In [1]:
from flask import Flask, render_template
import pandas as pd
import numpy as np
from queue import PriorityQueue

In [2]:
manifest_name = "SS TEST SHIP"
employee_name = "John Doe"
shipCase = 3

nrows=8
ncols=12

In [3]:
testManifest = ( pd.read_csv('./ship_cases/ShipCase'+str(shipCase)+'.txt',header=None,names=["col","row","weight","cont"]) )
testManifest['col'] = testManifest['col'].str.replace("\[| ","", regex=True)
testManifest['row'] = testManifest['row'].str.replace("\]| ","", regex=True)
testManifest['weight'] = testManifest['weight'].str.replace("{|}| ","", regex=True)
testManifest['cont'] = (testManifest['cont'].str[1:])

In [4]:
#creating 2d np array for ship
testManifestArray = np.array(testManifest.loc[:]).reshape(nrows,ncols,4)

In [5]:
#creating 2d np array for buffer
testBuffer = np.zeros(4*24).reshape(4,24)

####

In [6]:
def printState(array):
    for j in range(7,-1,-1):
        print("")
        for i in array[j]:
            if(i[3]=="UNUSED"):
                print("[  ]", end="\t")
            else:
                print(i[3], end="\t")
#printState(testManifestArray)

In [7]:
printState(testManifestArray)


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	Doe	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Ewe	Cow	Dog	Rat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	

### Defining a State

In [8]:
#Need Matrix of Ship Containers
#Need Matrix of Buffer Zone
#Need Col # the Crane is picking from
class State:
    def __init__(self, ship, topLayer=[], buffer=testBuffer):
        self.ship = ship
        self.buffer = buffer
        self.topLayer = topLayer

    def __str__(self):
        printState(self.ship)
        #return f"\n\nPick Col: {self.colPos}"
        return ""

In [9]:
#decorator clas for use with branching
class StateWrapper:
    def __init__(self, state, colPick=-1, h=40, moves = 0, cost=0, movesList = [], load=[], unload=[]):
        self.state = state
        self.colPick = colPick
        self.h = h
        self.moves = moves
        self.cost = cost
        self.movesList = movesList #of the form [(x1,y1), (x2,y2)]
        self.load = load
        self.unload = unload

    def __str__(self):
        printState(self.state.ship)
        t="\n\n"
        if self.colPick>0:
            t=f"\n\nPick Col: {self.colPick}"
        return t+f"\nh: {self.h}\n# of Moves: {self.moves}\nCost: {self.cost}\nMovesList: {self.movesList}\nLoad: {self.load}\nunload: {self.unload}"
    
    
    

In [10]:
print(StateWrapper(State(testManifestArray), 1))


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	Doe	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Ewe	Cow	Dog	Rat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	

Pick Col: 1
h: 40
# of Moves: 0
Cost: 0
MovesList: []
Load: []
unload: []


### Branching Factor is 12 (number of col)

In [11]:
#To branch we place our picked container (top most of selected col) on top of highest container of another col


#### Helper Function to find Highest Container in Stack

In [12]:
def stackHeight(array, col, nrows=nrows):
    for row in range(nrows-1,-1,-1):
        if(array[row,col,3] != "UNUSED"):
            return row
    return(-1)
    #-1 means no containers/blocked spaces in col

In [13]:
testCol=4
print(testManifestArray[:,testCol])
stackHeight(testManifestArray,testCol)

[['01' '05' '00000' 'UNUSED']
 ['02' '05' '00000' 'UNUSED']
 ['03' '05' '00000' 'UNUSED']
 ['04' '05' '00000' 'UNUSED']
 ['05' '05' '00000' 'UNUSED']
 ['06' '05' '00000' 'UNUSED']
 ['07' '05' '00000' 'UNUSED']
 ['08' '05' '00000' 'UNUSED']]


-1

In [14]:
def heights(array, cols=ncols):
    h = []
    for col in range(0,cols):
        h.append(stackHeight(array,col) + 1)
    return h

heights(testManifestArray)

[2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

#### Helper function to find top layer

In [15]:
def updateTop(array, cols=ncols):
    top = []
    for i in range(0,cols):
        h = stackHeight(array,i)
        if(h!=-1):
#             top.append( np.append(array[h,i,2:4],h) )
            top.append( (array[h,i]) )
        else:
#             top.append(np.array(['-99999','NONAME',h]))
            top.append(np.array(['00','00','-99999','NONAME']))
    return np.array(top)

# tmp = StateWrapper(State(testManifestArray, topLayer=updateTop(testManifestArray)), 1)
# print(tmp.state.topLayer)


In [16]:
def removeEmpty(array, pos = 3):
    filterArray = ((array[:,:,pos] != 'UNUSED') == (array[:,:,pos] != 'NAN'))
    return (array[filterArray])

#### Helper to find if can pick container

In [17]:
def pickable(array, col, nrows=nrows):
    h = stackHeight(array,col,nrows)
    if(array[h,col,3]=="NAN" or h==-1):
        return False
    return True

#### Place container to new col

In [18]:
def moveContainer(array, col, newCol, unload=0):
    if(col==newCol):
        return -1
    #find where to place container
    h1 = stackHeight(array,col)
    h2 = stackHeight(array,newCol) +1 
        
    newArray = array.copy()
    if(unload == 0):
        newArray[h2,newCol,2:4] = newArray[h1,col,2:4]  
    newArray[h1,col,2:4] = ['00000','UNUSED']
    
    return newArray

# print(State(testManifestArray))
# print()
# s1 = State(moveContainer(testManifestArray,4,0))
# s2 = State(moveContainer((s1.ship),4,3))
# s3 = State(moveContainer((s2.ship),4,5))

# print(s1)
# print(s2)


#### Cost of Moving from col x to col y

In [19]:
def costCol(array, curCol, newCol, empty=0, h1=-1, h2=-1):
    hs = heights(array)    
    
    if(h1<0):
        h1 = hs[curCol]
    if(h2<0):
        h2 = hs[newCol]
    
    #check if there exists in between area
    localMax=h2
    if(abs(curCol-newCol) > 1):
        localMax = max(hs[min(curCol,newCol)+1:max(curCol,newCol)+1]) - empty
    
    if(h1>=localMax):
        return 0 + abs(curCol-newCol) + abs(localMax-h2)
    else:
        return abs(localMax-h1) + abs(curCol-newCol) + abs(localMax-h2)
    

### Priority Queue Helper

In [20]:
#Code Chunk from official python priority queue page
from dataclasses import dataclass, field
from typing import Any

@dataclass(order=True)
class PrioritizedItem:
    priority: int
    cost: int
    item: Any=field(compare=False)

## Load Unload Problem

In [21]:
unload = []
load = []

#dummy data
unloadIn = [(1, 1), (1, 0), (2, 1)]
                    #dog        #cat
    
load = [['10000','first'],['20000','second'],['30000','third']]

# print(testManifestArray)

#### Load Function

In [22]:
#

#### Unload Function

In [23]:
#when tracking moves, compensate coordinates +1 for dropoff location to match pick up
    #**coordinates start at 1, not 0**
comp = 1

#initial state:
initialState = StateWrapper(State(testManifestArray),h=0,load=load,unload=unload)

unloadT = []

for i in unloadIn:
    unloadT.append(np.array(initialState.state.ship[i[0]-comp,i[1]-comp, 2:4]))
unloadT = np.array(unloadT)
unload = np.array(unloadT[(unloadT[:,1] != 'UNUSED') == (unloadT[:,1] != 'NAN')])

print(unload)
print(load)

[['10001' 'Ewe']
 ['09041' 'Cat']]
[['10000', 'first'], ['20000', 'second'], ['30000', 'third']]


In [30]:
def unloadTop(state, unload, cols=ncols):
    #Shows us cols of states to pick from which have exposed unload container
    top = np.array((updateTop(state.state.ship, cols=cols)))
    
    filterTop = top[((top[:,3] != 'NONAME') == (top[:,3] != 'NAN'))]
#     print(filterTop)
    
    unloadable = []
    for i in filterTop:
        for w in unload:
            if((i[2:4]==w).all()):
                unloadable.append(int(i[1])-1)
    
    return unloadable
    
t = unloadTop(initialState, unload)
print(t)
# print(State(initialState))
# print(State(t))

[0]


In [25]:
def countUnload(state, unload, cols=ncols):
    array = (state.state.ship)
    
    unloadable=[]
    for i in range(cols):
        num = 0
        for w in range(len(array[:,0])):
            if (array[w,i,3] in unload):
                num+=1
        unloadable.append(num)
    
    return np.array(unloadable)
    

#find nearest col to load from lest
def nearestLoad(unloadable):
    #expecting np.array
    for i in range(len(unloadable)):
        if(unloadable[i]==0):
            return i
        
x = countUnload(initialState, unload)
print(x)

print(nearestLoad(x))

def distToPink(row,col):
    return (nrows-row)+(col)

print(distToPink(0,0))
stackHeight(testManifestArray,3)

[2 0 0 0 0 0 0 0 0 0 0 0]
1
8


0

In [26]:
def loadHeuristic(state):
    load = state.load
    unload = state.unload
    return len(load)+len(unload)

In [38]:
old=100

#initial state:
initialState = StateWrapper(State(testManifestArray),h=0,load=load,unload=unload, colPick=0)


#duplicate states checker
duplicateState = []

#legal final states
finalState = []


#Create PriorityQueue for queing states
queueMain = PriorityQueue()
queue = PriorityQueue()

queueMain.put(PrioritizedItem(1, 1, initialState))
            
depth = 0
minCost = 10000


#using colPick attribute of class to show previous col

while not queueMain.empty():
    currentState=queueMain.get().item
    #At Each state we have 3 choices
        #unload available container                           (high priority)
        #load container from truck                            (high priority if crane empty)
        #move container on ship from on place to another      (low priority, only do if cant do prev 2)
    
    #1
    #attempt to unload container
    
    #first we find cols where there is a container we can pick right away
    print("TWESTEDTGETSETG\t\t\t\n", type(currentState.state.ship))
    unloadable = unloadTop(currentState, currentState.unload, cols=ncols)
    
    #we can branch here by choosing to pick each one and calculating h()
    if(len(unloadable)>0):
        ind=0
        for i in unloadable:
            
            #unload item
            removedShip = moveContainer(array=currentState.state.ship,col=i,newCol=0,unload=1)
            #find row removed from to calc h
            removedRow = stackHeight(currentState.state.ship,col=i)
            #create new unloadlist
            removedUnload = np.delete(currentState.unload,ind)
#TODO ADD COST FROM PREV POS TO CUR
            newRemovedState = StateWrapper(State(removedShip),cost=currentState.cost + distToPink(removedRow,i),movesList=currentState.movesList.copy(), load=currentState.load, unload=removedUnload, colPick=0)
            newRemovedState.h = loadHeuristic(newRemovedState)
            print("[UNLOADING] from col",i)
            if(newRemovedState not in duplicateState):
                duplicateState.append(newRemovedState)
                queueMain.put(PrioritizedItem(newRemovedState.h,newRemovedState.cost, newRemovedState))
                
            ind +=1
    else:
        print("Nothing available to unload")


#         #Stopping conditions

        depth+=1
        if((( depth>=1000 or len(finalState)>20) and len(finalState)!=0) or (queueMain.empty() and len(duplicateState)==old )):
            print("Exited at depth",depth, " and ", len(finalState), "final states", old,"=?",len(duplicateState))
            if(queueD.empty()):
                print("No More states to branch from!")
            break
        else:
            old = len(duplicateState)

TWESTEDTGETSETG			
 <class 'numpy.ndarray'>
[UNLOADING] from col 0
TWESTEDTGETSETG			
 <class 'int'>


TypeError: 'int' object is not subscriptable

In [31]:
print(initialState.state)
for i in duplicateState:
    print(i,"\n\n")


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	Doe	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Ewe	Cow	Dog	Rat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	



TypeError: 'int' object is not subscriptable

### Load/Unload Loop

## Balancing Problem

In [ ]:
#Function for testing balance
def balanceScore(array, cols=ncols):
    split=cols/2
    left = sum(((array[:,:int(split),2]).flatten()).astype(int))
    #print(left)
    right = sum(((array[:,int(split):,2]).flatten()).astype(int))
    #print(right)
    
    if(left==right):
        return 1
    if(left==0 or right==0):
        return 0
    
    return min(left,right)/max(left,right)
#balanceScore(testManifestArray)

In [ ]:
#Function for accepting balance
#default threshold of 10% 
def balanced(state, cols=ncols, thresh=.1):
    array = state.state.ship
    if(balanceScore(array, cols) > (1-thresh)):
        return True
    return False
#balanced(testManifestArray)

### Heuristic (Balance)

In [ ]:
def totalWeight(array):
    return sum(((array[:,:,2]).flatten()).astype(int))

def leftWeight(array, cols=ncols):
    split=cols/2
    return sum(((array[:,:int(split),2]).flatten()).astype(int))

def rightWeight(array, cols=ncols):
    split=cols/2
    return sum(((array[:,int(split):,2]).flatten()).astype(int))

def heavySide(state, cols=ncols):
    array = state.state.ship
    left = leftWeight(array,cols)
    right = rightWeight(array,cols)
    if(left < right):
        return 1
    return 0

def balanceHeuristicDroped(state, cols=ncols):
    #Given state, how balanced is it and what potential does it have?
    
    array = state.state.ship

    balanceMass = totalWeight(array)/2
    split = cols/2
    
    #which side is heavier
    side = heavySide(state)
        
    left = leftWeight(array,cols)
    right = rightWeight(array,cols)
    
    deficit = abs(balanceMass - max(left, right))

    if(side==0):
        weights = removeEmpty((array[:,:int(split)]))
    else:
        weights = removeEmpty((array[:,int(split):]))
    
    #list of cells sorted by weight
    sortedList = sorted(weights, key=lambda x: x[2], reverse=True)
    newDef = deficit
    h = 0
    for i in sortedList:
#         print("\ndef", newDef, "\n-")
#         print(i[2])

        if (newDef - int(i[2]) > 0):
            newDef -= int(i[2])
            h+=1
    
            
    #h is amount of containers needed to move to get close to balance
    return h, sortedList



In [ ]:
def balanceHeuristicPicked(state, colPick=-1, cols=ncols):
    #Picking from col pickCol, how good is this state?
    #what is the dist needed to drop into col on other side
        

    colPick = state.colPick
    array = state.state.ship
    
    side = heavySide(state)
    
    heightsArray = np.array(heights(array))
    
    #check if picking from heavy side    
    if(side==0 and colPick<cols/2):
        if( (heightsArray[int(cols/2):cols]<nrows-1).all() ):
#             print("side 0, moving to",cols/2)
            return cols/2 - colPick

        #there are full cols
#         print("left")
        for i in range(int(cols/2), cols):
            print(i)
            if(heightsArray[i] < nrows-1):
                return i - colPick
    
    if(side==1 and colPick>=cols/2):
        if( (heightsArray[int(cols/2)-1:-1]<nrows-1).all() ):
            return colPick - ((cols/2)-1)
        
        #there are full cols
#         print("right")
        for i in range(int(cols/2)-1, -1, -1):
            if(heightsArray[i] < nrows-1):
                return colPick-i
            
    #pulling from side that is lighter
    
    #h is slightly worse than worst case moving from heavier side
    return int(1 + cols/2)

In [ ]:
def sift(array, sortedList=0, cols = ncols):
    tmpArray = array.copy()
    cont= removeEmpty(array)
    containers = cont.copy()
    cont[:,2:4] = ['00000','UNUSED']
    
    
    filterArray = ((array[:,:,3] != 'UNUSED') == (array[:,:,3] != 'NAN'))    
    
    (tmpArray[filterArray]) = cont
    #make empty ship while keeping NAN's
    
    if(sortedList==0):
        sortedList = sorted(containers, key=lambda x: x[2], reverse=True)
    
    
    #Place all containers 1 by 1
    
    for i in range(len(sortedList)):
        height = (heights(tmpArray))        
        if(i%2 == 0):
            #every other time
            minRow = min(height[0:int(cols/2)])
            #place on left side
            for w in range(int(cols/2)-1,-1,-1):
                
                h = stackHeight(tmpArray,w)+1
                if(h == minRow):
                    #we can place container here
#                     print("\nwas", tmpArray[h,w])
                    tmpArray[h,w,2:4] = sortedList[i][2:4]
#                     print("\tchanged to ", tmpArray[h,w])
                    break
                    
        else:
            #every other time
            minRow = min(height[int(cols/2):cols+1])
            #place on right side
            for w in range(int(cols/2), cols+1):
                
                h = stackHeight(tmpArray,w)+1
                if(h == minRow):
                    #we can place container here
#                     print("\nwas", tmpArray[h,w])
                    tmpArray[h,w,2:4] = sortedList[i][2:4]
#                     print("\tchanged to ", tmpArray[h,w])
                    break
    
    return tmpArray

# print(State(sift(testManifestArray)))

# print(State(testManifestArray))
# s1 = StateWrapper(State(moveContainer(testManifestArray,4,7)))
# s2 = StateWrapper(State(moveContainer((s1.state.ship),3,4)))
# s3 = StateWrapper(State(moveContainer((s2.state.ship),5,3)))
# s4 = StateWrapper(State(moveContainer((s3.state.ship),2,6)))
# s5 = StateWrapper(State(moveContainer((s4.state.ship),1,5)))

# print(s1)
# print(s2)
# print(s3)
# print(s4)
# print(s5)

# if((s5.state.ship==sift(testManifestArray))).all():
#     print("samee")


In [ ]:
#check of possible to balance

def balancePossible(state) :
    array = state.state.ship
    cont = removeEmpty(array)
    sortedList = np.array(sorted(cont, key=lambda x: x[2], reverse=True))
    
    big = int(sortedList[0][2])
    other = sum((sortedList[1:,2].flatten()).astype(int))
#     print(big)
#     print(other)
    
    if((big>(big+other)/2) and (other/big <.9)):
        return False
    return True

# balancePossible(s1)

In [ ]:
def siftPick(state, col, siftState=sift(testManifestArray)):
    array = state.state.ship
    h = stackHeight(array,col)
    chosen = array[h,col]
    if((chosen == siftState[h,col])).all():
        return(20)

    for i in removeEmpty(siftState):
#         print((chosen, "vs", i))
        if((i[2:4] == chosen[2:4]).all()):
#             return abs(int(i[1])-int(chosen[1]))
            return(len(array))

# print(State(sift((testManifestArray))))
# print(s1)
# siftPick(s1,1)

In [ ]:
def siftDrop(state, siftState=sift(testManifestArray)):
    array = np.array(removeEmpty(state.state.ship))
    cont = np.array(removeEmpty(siftState))
    
    h=0
    
    for i in array:
        for w in cont:
            if((i==w).all()):
                h+=1
    
#     print(h)
    return len(array) - h
    
# print(State(sift((testManifestArray))))
# print(s2)
# siftDrop(s2)

### Branching Loop